In [0]:
%run "../jobs/reader"

In [0]:
%run "../jobs/data_quality"

In [0]:
%run "../jobs/transformer"

In [0]:
%run "../jobs/aggregator"

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType

spark = SparkSession.builder.getOrCreate()
reader = Reader(spark)
transformer = Transformer(spark)
dq = DataQuality(spark)
aggregator = Aggregator(spark)

def main():
    print("starting e-commerce-pipeline")

    # Input data paths for datasets
    order_path = "dbfs:/FileStore/shared_uploads/abhi687303@gmail.com/Test/Orders.json"
    customer_data = "dbfs:/FileStore/shared_uploads/abhi687303@gmail.com/Test/Customer.xlsx"
    product_data = "dbfs:/FileStore/shared_uploads/abhi687303@gmail.com/Test/Products.csv"

    # Schema definition for datasets-
    #---------customer dataset ------------------
    customer_schema = StructType([
    StructField("Customer ID", StringType(), True),
    StructField("Customer Name", StringType(), True),
    StructField("email", StringType(), True),
    StructField("phone", StringType(), True),
    StructField("address", StringType(), True),
    StructField("Segment", StringType(), True),
    StructField("Country", StringType(), True),
    StructField("City", StringType(), True),
    StructField("State", StringType(), True),
    StructField("Postal Code", StringType(), True),
    StructField("Region", StringType(), True)
    ])

    #----------product dataset schema ----------#
    product_schema = StructType([
    StructField("Product ID", StringType(), True),
    StructField("Category", StringType(), True),
    StructField("Sub-Category", StringType(), True),
    StructField("Product Name", StringType(), True),
    StructField("State", StringType(), True),
    StructField("Price per product", DoubleType(), True)
    ])

    #-------------Order dataset schema --------------- 
    order_schema = StructType([
        StructField("Row ID", IntegerType(), True),
        StructField("Order ID", StringType(), True),
        StructField("Order Date", StringType(), True),   
        StructField("Ship Date", StringType(), True),    
        StructField("Ship Mode", StringType(), True),
        StructField("Customer ID", StringType(), True),
        StructField("Product ID", StringType(), True),
        StructField("Quantity", IntegerType(), True),
        StructField("Price", DoubleType(), True),
        StructField("Discount", DoubleType(), True),
        StructField("Profit", DoubleType(), True)
    ])

    # Read input data 
    product_df = reader.read_csv(product_data,True, product_schema)
    customer_df = reader.read_excel(customer_data,True, customer_schema)
    order_df = reader.read_json(order_path,True, order_schema)

    clean_customer_df = dq.standardize_column_names(customer_df)
    clean_order_df = dq.standardize_column_names(order_df)
    clean_product_df = dq.standardize_column_names(product_df)

    # ---- Save raw tables ------------
    # clean_product_df.write.format("delta").saveAsTable("product")
    # clean_customer_df.write.format("delta").saveAsTable("customer")
    # clean_order_df.write.format("delta").saveAsTable("order")

    # ---------------data quality checks on customer dataset---------------------
    valid_cust_df = dq.filter_not_null(clean_customer_df,["customer_id","country"])
    valid_cust_df = dq.clean_name_column(valid_cust_df,"customer_name")
    # valida_cust_df.show(2)

    # ----------------data quality checks on order dataset-------------------------
    
    valid_order_df = dq.filter_not_null(clean_order_df,["order_id","order_date","ship_date","customer_id","product_id","quantity",
                                                        "price","discount","profit"])
    valid_order_df = dq.filter_positive_values(valid_order_df,["quantity","price"])
    valid_order_df = dq.filter_valid_orders(valid_order_df,"order_date","ship_date")
    # valid_order_df.show(2)

    #----------------data quality checks on product dataset --------------------
    valid_product_df = dq.filter_not_null(clean_product_df,["product_id","category","sub_category","product_name","price_per_product"])
    valid_product_df = dq.filter_positive_values(clean_product_df,["price_per_product"])

    #---------- Enrich order- with joins to get all details at one place for aggregation-----------
    valid_order_df = transformer.transform_orders(valid_order_df)
    valid_cust_df = valid_cust_df.withColumn("customer_id", F.trim(F.col("customer_id")))
    valid_order_df = valid_order_df.withColumn("customer_id", F.trim(F.col("customer_id")))
    enrich_orders = transformer.enrich_orders(valid_order_df,valid_cust_df,valid_product_df)

    # ----------------save enrich table for sql queries--------------
    # enrich_orders.write.format("delta").saveAsTable("enriched_orders")

    #------------ Get final results results ------------------
    agg_results = aggregator.aggregate_profit_by_year(enrich_orders)
    print("final aggregated data")
    agg_results.show(10)
    print("enriched columns :",enrich_orders.columns)
    agg_profit_cat = aggregator.aggregate_profit(enrich_orders)

    # save aggregated table by categories
    # agg_profit_cat.write.format("delta").saveAsTable("aggregated_table")
    print("aggregated profit by category :")
    agg_profit_cat.show(5)


    #--- Aggregation results using sqls---------------
    print("----------Profit by Year------------------------------")
    spark.sql("SELECT year,round(SUM(profit),2) AS total_profit FROM enriched_orders GROUP BY year ORDER BY year").show(5)

    print("----------Profit by Year + Product Category------------")
    spark.sql("""SELECT year,category,round(SUM(profit),2) AS total_profit FROM enriched_orders
            GROUP BY year, category
            --ORDER BY year, category
            limit 5""").show(5)

    print("--------------Profit by Customer-------------------")
    spark.sql("""SELECT year, customer_id,customer_name, round(SUM(profit),2) AS total_profit FROM enriched_orders
            GROUP BY year, customer_id, customer_name
            ORDER BY year
                limit 10""").show(5)
    
    print("--------------Profit by Customer + Year-------------")
    spark.sql("""SELECT year, customer_id,customer_name,round(SUM(profit),2) AS total_profit FROM enriched_orders
            GROUP BY year, customer_id, customer_name
            ORDER BY year
            limit 10;""").show(5)
    

if __name__ == "__main__":
    main()

starting e-commerce-pipeline
final aggregated data
+----+----------+
|year|profit_sum|
+----+----------+
|2015|  47304.51|
|2014|  31687.24|
|2016|  61267.36|
|2017| 106333.12|
+----+----------+

enriched columns : ['order_id', 'order_date', 'ship_date', 'customer_id', 'customer_name', 'country', 'product_id', 'category', 'sub_category', 'profit', 'year']
aggregated profit by category :
+----+---------------+------------+-----------+--------------+----------+
|year|       category|Sub_Category|customer_id| customer_name|profit_sum|
+----+---------------+------------+-----------+--------------+----------+
|2014|      Furniture| Furnishings|   AP-10915|Arthur Prichep|     54.47|
|2015|Office Supplies|     Binders|   RP-19855|      Roy Phan|      26.6|
|2017|Office Supplies|         Art|   TA-21385|  Tom Ashbrook|      2.04|
|2016|      Furniture|      Chairs|   EB-14110|Eugene Barchas|    -21.72|
|2017|     Technology|      Phones|   NG-18355|    Nat Gilpin|     18.34|
+----+------------

In [0]:
# dbutils.fs.rm("dbfs:/user/hive/warehouse/product", recurse=True)
# dbutils.fs.rm("dbfs:/user/hive/warehouse/customer", recurse=True)
# dbutils.fs.rm("dbfs:/user/hive/warehouse/order", recurse=True)
#dbutils.fs.rm("dbfs:/user/hive/warehouse/enriched_orders", recurse=True)

True